## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import re

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import classification_report

np.random.seed(13)
print(sklearn.__version__)
print(np.__version__)
print(pd.__version__)

1.6.1
2.0.2
2.2.3


## Import data

In [2]:
df = pd.read_csv("../data/JFP Credential Data_All_Journals_Final_1.2025.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5630 entries, 0 to 5629
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         5630 non-null   object
 1   Abstract      4046 non-null   object
 2   Authors       5466 non-null   object
 3   documentType  5630 non-null   object
 4   pub_month     5337 non-null   object
 5   pub_year      5630 non-null   int64 
 6   pubtitle      5630 non-null   object
 7   subjectTerms  4067 non-null   object
 8   Database      5630 non-null   object
dtypes: int64(1), object(8)
memory usage: 396.0+ KB


## Extract Credentials

In [4]:
df["Credentials"] = (df
                     .Authors
                     .apply(lambda x: re.findall(r"(CFP|PhD|CFA|JD|CPA|CLU|ChFC|LLM|AIF|CRC)", str(x)))
                     .apply(lambda x: np.nan if len(x) == 0 else x)
                     )

## Group Credentials into target groups

In [5]:
def target_grouping(x):
    if type(x) != list:
        return np.nan
    x = set(x)
        
    if "PhD" in set(x) and len(set(x)) == 1:
        return "Academic"
    elif "PhD" in set(x) and len(set(x)) > 1:
        return "Both"
    else:
        return "Practitioner"

df["target_grouping"] = df.Credentials.apply(target_grouping)

## Drop nulls

In [6]:
df = df.dropna(subset=["target_grouping", "Abstract"]).reset_index(drop=True)

In [7]:
df.head()

,Title,Abstract,Authors,documentType,pub_month,pub_year,pubtitle,subjectTerms,Database,Credentials,target_grouping
0,How Do Commodities Fit into Client Portfolios?,The standard stock--bond portfolio mix remains...,"Fink, Jason D, PhD;Fink, Kristin E, PhD",Commentary,Oct,2022,Journal of Financial Planning,"Financial planning , Investment , Bonds , Sto...",ProQuest One Academic,"[PhD, PhD]",Academic
1,The Role of Financial Planners on African Amer...,"* Addressing the issue of ""wealth gap,"" partic...","Chen, Leon, PhD, FRM;Duffy, Sophia, JD, CPA;Hi...",Commentary,Nov,2022,Journal of Financial Planning,"Credit , Small business , Wealth distribution...",ProQuest One Academic,"[PhD, JD, CPA, PhD, CFP]",Both
2,"How to Effectively Manage Six ""Oh Bleep"" Clien...",Managing difficult client situations is part o...,"Heye, Chris, PhD",Journal Article,May,2022,Journal of Financial Planning,"Interest rates , Financial management , Finan...",ProQuest One Academic,[PhD],Academic
3,Inflation Uncertainty Calls for Portfolio Prot...,"Inflation currently sits at 7.9 percent, havin...","Marotta, Ryann, CFP®, CFA",Journal Article,May,2022,Journal of Financial Planning,"Investors , Investments , Pandemics , Monetar...",ProQuest One Academic,"[CFP, CFA]",Practitioner
4,Financial Planners' Gray Divorce Checklist,Divorce today is less common for adults younge...,"Stephenson, Angie M, CFP®, CPA/PFS",Journal Article,May,2022,Journal of Financial Planning,"Divorce , Marital separation , Marriage , Tax...",ProQuest One Academic,"[CFP, CPA]",Practitioner


In [8]:
X = df.Abstract
y = df.target_grouping

In [9]:
X.shape, y.shape

((2608,), (2608,))

In [10]:
y.value_counts()

target_grouping
Practitioner    1620
Both             570
Academic         418
Name: count, dtype: int64

In [11]:
y.value_counts().to_csv("../results/multiclass_target_distribution.csv")

## Encode Target

In [12]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [13]:
encoder.classes_

array(['Academic', 'Both', 'Practitioner'], dtype=object)

## Create Pipeline

In [14]:
models = [LogisticRegression(class_weight="balanced"), 
          DecisionTreeClassifier(class_weight="balanced"), 
          RandomForestClassifier(class_weight="balanced"),
          HistGradientBoostingClassifier(class_weight="balanced")]

# Define the custom transformer
class DenseTransformer:
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # make the array dense
        X = X.toarray()
        return X

pipelines = [Pipeline([
    ("vectorizer", TfidfVectorizer()),
    ("densify", DenseTransformer()),
    ("classifier", model)
    ]) for model in models]

In [15]:
pipelines

[Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                 ('densify',
                  <__main__.DenseTransformer object at 0x7f88701488f0>),
                 ('classifier', LogisticRegression(class_weight='balanced'))]),
 Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                 ('densify',
                  <__main__.DenseTransformer object at 0x7f886db6b680>),
                 ('classifier',
                  DecisionTreeClassifier(class_weight='balanced'))]),
 Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                 ('densify',
                  <__main__.DenseTransformer object at 0x7f886db6b470>),
                 ('classifier',
                  RandomForestClassifier(class_weight='balanced'))]),
 Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                 ('densify',
                  <__main__.DenseTransformer object at 0x7f886db69d00>),
                 ('classifier',
                  HistGradientBoostingClassifier(class_weight='balan

## Cross Validation

In [19]:
results = [cross_validate(pipeline, X, y, scoring=["precision_weighted", "recall_weighted", "accuracy", "f1_weighted"], cv=5, n_jobs=-1) for pipeline in pipelines]

In [22]:
result_df = pd.DataFrame({
    "model": models,
    "avg_precision_weighted": [result["test_precision_weighted"].mean() for result in results],
    "avg_recall_weighted": [result["test_recall_weighted"].mean() for result in results],
    "avg_f1_weighted": [result["test_f1_weighted"].mean() for result in results],
    "avg_accuracy": [result["test_accuracy"].mean() for result in results]
})

In [32]:
result_df.round(3)

,model,avg_precision_weighted,avg_recall_weighted,avg_f1_weighted,avg_accuracy
0,LogisticRegression(class_weight='balanced'),0.703,0.696,0.698,0.696
1,DecisionTreeClassifier(class_weight='balanced'),0.576,0.570,0.572,0.570
2,"(DecisionTreeClassifier(max_features='sqrt', r...",0.650,0.662,0.572,0.662
3,HistGradientBoostingClassifier(class_weight='b...,0.668,0.692,0.673,0.692


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [25]:
print(f"Train set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

Train set shape: (1956,)
Test set shape: (652,)


In [26]:
lr_pipeline = pipelines[0]

lr_pipeline.fit(X_train, y_train)

y_pred = lr_pipeline.predict(X_test)

lr_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, target_names=encoder.classes_))

print("Logistic Regression Classification Report")
lr_report


Logistic Regression Classification Report


,Academic,Both,Practitioner,accuracy,macro avg,weighted avg
precision,0.504673,0.496855,0.852332,0.708589,0.617953,0.718924
recall,0.514286,0.556338,0.812346,0.708589,0.627656,0.708589
f1-score,0.509434,0.524917,0.831858,0.708589,0.622070,0.713085
support,105.000000,142.000000,405.000000,0.708589,652.000000,652.000000


In [27]:
dt_pipeline = pipelines[1]

dt_pipeline.fit(X_train, y_train)

y_pred = dt_pipeline.predict(X_test)

dt_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, target_names=encoder.classes_))

print("Decision Tree Classification Report")
dt_report

Decision Tree Classification Report


,Academic,Both,Practitioner,accuracy,macro avg,weighted avg
precision,0.300000,0.327586,0.714286,0.547546,0.447291,0.563348
recall,0.285714,0.401408,0.666667,0.547546,0.451263,0.547546
f1-score,0.292683,0.360759,0.689655,0.547546,0.447699,0.554095
support,105.000000,142.000000,405.000000,0.547546,652.000000,652.000000


In [28]:
rf_pipeline = pipelines[2]

rf_pipeline.fit(X_train, y_train)

y_pred = rf_pipeline.predict(X_test)

rf_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, target_names=encoder.classes_))

print("Random Forest Classification Report")
rf_report

Random Forest Classification Report


,Academic,Both,Practitioner,accuracy,macro avg,weighted avg
precision,1.000000,0.588235,0.672850,0.670245,0.753695,0.707107
recall,0.076190,0.211268,0.985185,0.670245,0.424214,0.670245
f1-score,0.141593,0.310881,0.799599,0.670245,0.417358,0.587193
support,105.000000,142.000000,405.000000,0.670245,652.000000,652.000000


In [30]:
gbt_pipeline = pipelines[3]

gbt_pipeline.fit(X_train, y_train)

y_pred = gbt_pipeline.predict(X_test)

gbt_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, target_names=encoder.classes_))

print("Gradient Boosted Trees Classification Report")
gbt_report

Gradient Boosted Trees Classification Report


,Academic,Both,Practitioner,accuracy,macro avg,weighted avg
precision,0.547945,0.551724,0.781857,0.714724,0.627176,0.694066
recall,0.380952,0.450704,0.893827,0.714724,0.575161,0.714724
f1-score,0.449438,0.496124,0.834101,0.714724,0.593221,0.698546
support,105.000000,142.000000,405.000000,0.714724,652.000000,652.000000


## Save Results

In [31]:
result_df.to_csv("../results/multiclass_baseline_model_results.csv", index=False)